In [17]:
import sys
!{sys.executable} -m pip install datasets pandas scikit-learn synthcity

  Using cached datasets-2.13.1-py3-none-any.whl (486 kB)
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
  Using cached xxhash-3.2.0-cp310-cp310-macosx_11_0_arm64.whl (31 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.7 MB/s eta 0:00:00m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 7.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
  Using cached multiprocess-0.70.14-py310-none-any.whl (134 kB)


In [18]:
from sklearn.datasets import load_diabetes
X, y = load_diabetes(return_X_y=True, as_frame=True)
X["target"] = y

In [6]:
from synthcity.plugins.core.dataloader import GenericDataLoader
loader = GenericDataLoader(
    X,
    target_column="target",
    sensitive_columns=["sex"],
)

In [7]:
from synthcity.plugins import Plugins
Plugins().list()

[2023-07-24T12:16:03.473054+0200][23284][CRITICAL] module disabled: /Users/petersk/anaconda3/envs/synthesizers/lib/python3.10/site-packages/synthcity/plugins/generic/plugin_goggle.py


['dpgan',
 'survival_nflow',
 'bayesian_network',
 'ctgan',
 'survae',
 'nflow',
 'privbayes',
 'timegan',
 'rtvae',
 'radialgan',
 'image_adsgan',
 'survival_gan',
 'image_cgan',
 'fflows',
 'timevae',
 'pategan',
 'adsgan',
 'decaf',
 'tvae',
 'survival_ctgan',
 'ddpm']

In [8]:
syn_model = Plugins().get("adsgan")
syn_model.fit(loader)

[2023-07-24T12:17:49.470617+0200][23284][CRITICAL] module disabled: /Users/petersk/anaconda3/envs/synthesizers/lib/python3.10/site-packages/synthcity/plugins/generic/plugin_goggle.py
  7%|█████████                                                                                                                | 749/10000 [00:37<07:47, 19.77it/s]


In [37]:
df = syn_model.generate(count=10).dataframe()

In [38]:
from datasets import Dataset
ds = Dataset.from_pandas(df).remove_columns(['__index_level_0__'])

In [39]:
df, ds

(        age       sex       bmi        bp        s1        s2        s3  \
 0  0.040529  0.050680 -0.023974 -0.024673 -0.074178 -0.056505  0.018165   
 1 -0.052546 -0.044642 -0.031749  0.032689  0.030035  0.034626  0.029267   
 2 -0.002148 -0.044642 -0.024803  0.038914 -0.030641 -0.027864  0.029196   
 3  0.041746 -0.044642  0.046440 -0.031589  0.060845 -0.010458 -0.010906   
 4  0.034744 -0.044642 -0.026004 -0.020371 -0.031106 -0.043824  0.066206   
 5 -0.061096 -0.044642 -0.067477 -0.022831 -0.010330 -0.003036  0.011049   
 6 -0.039588  0.050680 -0.076333 -0.039095 -0.020957 -0.059864  0.104715   
 7 -0.040120 -0.044642 -0.060148 -0.045673 -0.064324 -0.079522  0.046909   
 8 -0.046189 -0.044642  0.008232 -0.061132 -0.021850 -0.038516  0.012202   
 9  0.033125 -0.044642 -0.025512  0.094885  0.050642  0.035066  0.009034   
 
          s4        s5        s6      target  
 0 -0.033802  0.002838 -0.035455  229.668805  
 1 -0.000517 -0.041243 -0.027160   71.745109  
 2  0.023852 -0.02604

In [40]:
from synthcity.utils.serialization import load, load_from_file, save, save_to_file
buff = save(syn_model)
type(buff)

bytes

In [42]:
model = load(buff)
model.generate(count=10).dataframe()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.041928,0.050680,-0.049977,-0.041555,-0.059076,-0.064916,0.080995,-0.076395,-0.042025,-0.041905,37.902568
1,0.025877,0.050680,-0.027903,0.005623,-0.024548,0.007161,-0.046115,0.058236,-0.001508,0.004889,85.662816
2,0.075615,0.050680,-0.030687,0.000701,0.015511,0.022315,0.002358,-0.051799,-0.012025,0.027687,49.619880
3,0.033887,0.050680,-0.031521,0.045810,0.041755,0.067904,0.046702,0.034927,-0.010333,-0.012505,60.554912
4,0.056510,-0.044642,-0.011124,0.057063,-0.016927,0.006099,-0.002101,-0.005252,0.022336,-0.015430,249.175813
5,-0.056629,-0.044642,-0.036762,-0.030477,-0.036695,-0.053701,0.001344,-0.042633,-0.027888,-0.065674,63.365340
6,0.014531,-0.044642,-0.039893,-0.057257,-0.032206,-0.032329,0.181179,-0.076395,-0.063118,-0.040649,50.413358
7,-0.045434,0.050680,-0.041318,-0.024585,-0.004572,-0.028128,0.008952,-0.005517,0.030611,0.077768,158.327852
8,0.012847,-0.044642,-0.037868,-0.038084,0.042235,-0.027002,0.066968,0.052176,0.018336,-0.028194,76.986612
9,-0.038447,0.050680,-0.030454,-0.003259,-0.021955,-0.021275,-0.028441,0.029623,0.064685,0.051705,154.346543


In [44]:
from datasets import load_dataset
ds = load_dataset("mstz/breast")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset breast downloaded and prepared to /Users/petersk/.cache/huggingface/datasets/mstz___breast/cancer/1.0.0/2975d6eb7a99f013c56d50c583f164612c532b3fc703db713d5b19fbb2f19804. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [55]:
from pandas import DataFrame
df = DataFrame(ds['train'])
df

,clump_thickness,uniformity_of_cell_size,uniformity_of_cell_shape,marginal_adhesion,single_epithelial_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,is_cancer
0,5,1,1,1,2,1,3,1,1,0
1,5,4,4,5,7,10,3,2,1,0
2,3,1,1,1,2,2,3,1,1,0
3,6,8,8,1,3,4,3,7,1,0
4,4,1,1,3,2,1,3,1,1,0
...,...,...,...,...,...,...,...,...,...,...
678,3,1,1,1,3,2,1,1,1,0
679,2,1,1,1,2,1,1,1,1,0
680,5,10,10,3,7,3,8,10,2,1
681,4,8,6,4,3,4,10,6,1,1


In [56]:
loader = GenericDataLoader(
    df,
    target_column="is_cancer",
    sensitive_columns=[],
)
breast_model = Plugins().get("adsgan")
breast_model.fit(loader)

[2023-07-24T12:39:45.595664+0200][23284][CRITICAL] module disabled: /Users/petersk/anaconda3/envs/synthesizers/lib/python3.10/site-packages/synthcity/plugins/generic/plugin_goggle.py
  6%|███████▏                                                                                                                 | 599/10000 [00:52<13:40, 11.46it/s]


In [57]:
breast_model.generate(count=10).dataframe()

,clump_thickness,uniformity_of_cell_size,uniformity_of_cell_shape,marginal_adhesion,single_epithelial_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,is_cancer
0,1,1,1,1,1,1,1,1,1,0
1,10,4,3,10,6,10,7,5,1,1
2,4,1,1,1,1,1,2,1,1,0
3,10,6,5,10,6,10,2,1,1,1
4,10,4,6,4,2,5,4,9,2,1
5,1,1,1,1,2,1,3,1,1,0
6,3,3,8,6,10,10,5,1,1,0
7,1,1,3,1,2,1,2,1,1,0
8,6,10,3,10,6,10,5,1,1,1
9,1,1,3,2,2,1,3,1,1,0
